In [ ]:
from flask import Flask, render_template
import paho.mqtt.client as mqtt
import uuid

client_idd = "mqttx_" + str(uuid.uuid4())
app = Flask(__name__)

# MQTT Broker details
broker_address = "broker.emqx.io"
mqtt_port = 1883
mqtt_topic = "StressDet092111161173"  # Topic to subscribe to

# Initialize list to store readings
readings = []

# Create a MQTT client instance
client = mqtt.Client(client_id=client_idd, clean_session=True, callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# Callbacks for successful connection and disconnection
def on_mqtt_connect(client, userdata, flags, rc, ir):
    print("Connected to MQTT broker with result code " + str(rc))
    # Subscribe to the topic
    client.subscribe(mqtt_topic)

def on_message(client, userdata, msg):
    print("Received message: " + msg.payload.decode())
    # Update readings list with received data
    update_readings(msg.payload.decode())

# Assign the callback functions
client.on_connect = on_mqtt_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect(broker_address, mqtt_port, 60)

# Function to update readings list with received data
def update_readings(data):
    global readings
    data_parts = data.split(", ")
    parsed_data = {}
    for part in data_parts:
        key, value = part.split(": ")
        parsed_data[key.strip()] = value.strip()
    readings.append(parsed_data)

# Flask route to render the HTML template
@app.route('/')
def index():
    return render_template('index.html', readings=readings)

# Run the Flask application
if __name__ == '__main__':
    client.loop_start()  # Start the MQTT client loop
    app.run(debug=True, port=3333, use_reloader=False)


Received message: Heart Rate: 88.24, Temperature: 26.0, Stress: NO STRESS
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:3333
Press CTRL+C to quit


Connected to MQTT broker with result code Success
Received message: Heart Rate: 88.24, Temperature: 26.0, Stress: NO STRESS
Received message: Heart Rate: 88.24, Temperature: 26.0, Stress: NO STRESS


127.0.0.1 - - [03/May/2024 13:29:29] "GET / HTTP/1.1" 200 -


Received message: Heart Rate: 88.24, Temperature: 26.0, Stress: NO STRESS
Received message: Heart Rate: 88.24, Temperature: 26.0, Stress: NO STRESS
